# 嵌入
* 将文本的小部分转换为数字（向量），以便可以轻松存储和通过向量数据库进行搜索。

## 创建你的 .env 文件
* 在 GitHub 仓库中，我们包含了一个名为 .env.example 的文件
* 将该文件重命名为 .env 文件，你将在这里添加你的机密 API 密钥。记得包括：
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

我们将把我们的LangSmith项目称为 **003-embeddings**。

## 跟踪操作
从现在起，我们可以跟踪这个项目在 LangSmith 上的操作 **和成本**：
* [smith.langchain.com](https://smith.langchain.com)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

## 提醒：RAG 过程的步骤。
* 当您加载文档时，最终会得到字符串。有时字符串的大小会超出上下文窗口。在这些情况下，我们将使用 RAG 技术：
    * 将文档分割成小块。
    * **将文本块转换为数值块（嵌入）**。
    * 将嵌入加载到向量数据库（即向量存储）中。
    * 加载问题并检索最相关的嵌入以进行回答。
    * 将嵌入发送给 LLM 以正确格式化响应。

## 分隔符：将加载的文档划分为小块文本
* 也称为“文档转换器”。
* 请参阅文档页面 [这里](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)。
* 请参阅内置分隔符的列表 [这里](https://python.langchain.com/v0.1/docs/integrations/document_transformers/)。

#### 按字符简单拆分

In [34]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/be-good.txt")

loaded_data = loader.load()

In [1]:
loaded_data

In [2]:
loaded_data[0].page_content

In [35]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [47]:
texts = text_splitter.create_documents([loaded_data[0].page_content])

In [53]:
len(texts)

2

In [48]:
texts[0]

Document(page_content='Be good')

In [54]:
# texts[1]

带元数据的拆分

In [55]:
metadatas = [{"chunk": 0}, {"chunk": 1}]

documents = text_splitter.create_documents(
    [loaded_data[0].page_content, loaded_data[0].page_content], 
    metadatas=metadatas
)

In [58]:
documents[0]

Document(page_content='Be good', metadata={'chunk': 0})

In [59]:
print(documents[0])

page_content='Be good' metadata={'chunk': 0}


## 嵌入：将文本块转换为数字块
* 向量数据库与数字的处理速度非常快。
* 请参见[这里](https://python.langchain.com/v0.1/docs/modules/data_connection/text_embedding/)的文档页面。
* 请在[这里](https://python.langchain.com/docs/integrations/text_embedding/)查看嵌入模型提供者的列表。
* **LangChain中的基础嵌入类提供了两种方法：一种用于嵌入文档，另一种用于嵌入查询**。

In [60]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [72]:
chunks_of_text =     [
        "Hi there!",
        "Hello!",
        "What's your name?",
        "Bond, James Bond",
        "Hello Bond!"
    ]

In [73]:
embeddings = embeddings_model.embed_documents(chunks_of_text)

In [69]:
len(embeddings)

5

In [70]:
len(embeddings[0])

1536

In [74]:
print(embeddings[0][:5])

[-0.020291371271014214, -0.007072774693369865, -0.022869061678647995, -0.02623402513563633, -0.03748443350195885]


In [75]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

In [76]:
len(embedded_query)

1536